In [5]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3")

state_size = env.observation_space.n
action_size = env.action_space.n

print("Espace des états:", state_size)
print("Espace des actions:", action_size)

policy_table = np.ones((state_size, action_size)) / action_size

value_table = np.zeros(state_size)

print("Policy table (premières lignes):")
print(policy_table[:5])
print("\nValue table (premières lignes):")
print(value_table[:5])

Espace des états: 500
Espace des actions: 6
Policy table (premières lignes):
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Value table (premières lignes):
[0. 0. 0. 0. 0.]


In [2]:
for episode in range(20):
    state = env.reset()[0]
    done = False
    total_reward = 0
    
    print(f"\nÉpisode {episode + 1}")
    print("---------------------")
    
    while not done:
        # Action aléatoire selon la politique
        action = np.random.choice(action_size, p=policy_table[state])
        next_state, reward, done, truncated, info = env.step(action)
        
        print(f"État: {state}, Action: {action}, Récompense: {reward}")
        total_reward += reward
        state = next_state
        
        if done or truncated:
            break
    
    print(f"Récompense totale: {total_reward}")


Épisode 1
---------------------
État: 114, Action: 2, Récompense: -1
État: 134, Action: 5, Récompense: -10
État: 134, Action: 1, Récompense: -1
État: 34, Action: 4, Récompense: -10
État: 34, Action: 5, Récompense: -10
État: 34, Action: 5, Récompense: -10
État: 34, Action: 2, Récompense: -1
État: 34, Action: 4, Récompense: -10
État: 34, Action: 4, Récompense: -10
État: 34, Action: 4, Récompense: -10
État: 34, Action: 2, Récompense: -1
État: 34, Action: 5, Récompense: -10
État: 34, Action: 0, Récompense: -1
État: 134, Action: 5, Récompense: -10
État: 134, Action: 5, Récompense: -10
État: 134, Action: 3, Récompense: -1
État: 114, Action: 4, Récompense: -10
État: 114, Action: 1, Récompense: -1
État: 14, Action: 4, Récompense: -10
État: 14, Action: 4, Récompense: -10
État: 14, Action: 5, Récompense: -10
État: 14, Action: 4, Récompense: -10
État: 14, Action: 3, Récompense: -1
État: 14, Action: 2, Récompense: -1
État: 34, Action: 5, Récompense: -10
État: 34, Action: 2, Récompense: -1
État: 3

In [3]:
# Paramètres
learning_rate = 0.01
gamma = 0.99
epsilon = 0.2 
num_episodes = 1000

for episode in range(num_episodes):
    state = env.reset()[0]
    done = False
    episode_states = []
    episode_actions = []
    episode_rewards = []
    
    while not done:
        action_probs = policy_table[state]
        action = np.random.choice(action_size, p=action_probs)
        next_state, reward, done, truncated, info = env.step(action)
        
        episode_states.append(state)
        episode_actions.append(action)
        episode_rewards.append(reward)
        
        state = next_state
        if done or truncated:
            break
    
    discounted_rewards = []
    running_add = 0
    for r in reversed(episode_rewards):
        running_add = r + gamma * running_add
        discounted_rewards.insert(0, running_add)
    
    discounted_rewards = np.array(discounted_rewards)
    discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)
    
    # Mise à jour de la politique et des valeurs
    for i in range(len(episode_states)):
        state = episode_states[i]
        action = episode_actions[i]
        advantage = discounted_rewards[i] - value_table[state]
        
        # Ancienne probabilité
        old_prob = policy_table[state][action]
        
        # Calcul du ratio
        ratio = policy_table[state][action] / old_prob
        
        # Clipping
        clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
        
        # Mise à jour de la politique
        policy_table[state][action] += learning_rate * min(ratio * advantage, clipped_ratio * advantage)
        
        # Renormalisation des probabilités
        policy_table[state] = np.clip(policy_table[state], 1e-8, 1 - 1e-8)
        policy_table[state] = policy_table[state] / np.sum(policy_table[state])
        
        # Mise à jour de la valeur
        value_table[state] += learning_rate * (discounted_rewards[i] - value_table[state])



In [4]:
num_eval_episodes = 20
total_rewards = []

for episode in range(num_eval_episodes):
    state = env.reset()[0]
    done = False
    total_reward = 0
    
    while not done:
        action = np.argmax(policy_table[state])  # Meilleure action selon la politique
        state, reward, done, truncated, info = env.step(action)
        total_reward += reward
        if done or truncated:
            break
    
    total_rewards.append(total_reward)
    print(f"Épisode {episode + 1}: Récompense totale = {total_reward}")

mean_reward = np.mean(total_rewards)
print(f"\nRécompense moyenne sur {num_eval_episodes} épisodes: {mean_reward}")



Épisode 1: Récompense totale = -200
Épisode 2: Récompense totale = -200
Épisode 3: Récompense totale = -2000
Épisode 4: Récompense totale = -2000
Épisode 5: Récompense totale = -200
Épisode 6: Récompense totale = -2000
Épisode 7: Récompense totale = -200
Épisode 8: Récompense totale = -200
Épisode 9: Récompense totale = -200
Épisode 10: Récompense totale = -200
Épisode 11: Récompense totale = -200
Épisode 12: Récompense totale = -2000
Épisode 13: Récompense totale = -200
Épisode 14: Récompense totale = -1973
Épisode 15: Récompense totale = -200
Épisode 16: Récompense totale = -2000
Épisode 17: Récompense totale = -200
Épisode 18: Récompense totale = -200
Épisode 19: Récompense totale = -1982
Épisode 20: Récompense totale = -200

Récompense moyenne sur 20 épisodes: -827.75
